In [17]:
import pandas as pd
import numpy as np
# import xgboost as xgb
from sklearn import preprocessing, cross_validation, neighbors, svm
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, VotingClassifier, GradientBoostingClassifier, ExtraTreesClassifier, \
  RandomForestRegressor, AdaBoostRegressor, RandomForestRegressor, GradientBoostingRegressor, AdaBoostClassifier
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LogisticRegression, LinearRegression

import pickle
DIR_DATA = "data"
DIR_PROCESSED = "processed"
LABEL = "rating"
NON_PREDICTORS = [LABEL]#, "name","anime_id"]
CV_FOLDS = 5


In [18]:
# Anime.csv

# anime_id - myanimelist.net's unique id identifying an anime.
# name - full name of anime.
# genre - comma separated list of genres for this anime.
# type - movie, TV, OVA, etc.
# rating - average rating out of 10 for this anime.
# members - number of community members that are in this anime's "group".


# Rating.csv

# user_id - non identifiable randomly generated user id.
# anime_id - the anime that this user has rated.
# rating - rating out of 10 this user has assigned (-1 if the user watched it but didn't assign a rating).


In [19]:
animes = pd.read_csv(DIR_DATA + '/anime.csv')
ratings = pd.read_csv(DIR_DATA + '/rating.csv')

label = animes.rating
labeled_ratings = ratings[ratings.rating != -1][:1000]

# TODO: Get things with similar names classified together


In [20]:
animes.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [21]:
ratings.tail()

,user_id,anime_id,rating
7813732,73515,16512,7
7813733,73515,17187,9
7813734,73515,22145,10
7813735,73516,790,9
7813736,73516,8074,9


In [22]:
# TODO: Get user_id, previous rankings

In [23]:
def predict_naive(ratings, animes):
  prediction = []
  for anime_id in ratings.anime_id:
    prediction.append(round(animes[animes.anime_id == anime_id].rating.item()))
  return prediction

In [28]:
predictions = predict_naive(ratings,animes)

ValueError: can only convert an array of size 1 to a Python scalar

In [25]:
animes[animes.anime_id == 8074].rating

1709    7.46
Name: rating, dtype: float64

In [ ]:
print(mean_absolute_error(predictions, ratings.rating.tolist()))


In [11]:
animes.type = animes.type.astype('category')


In [12]:
animes["genre_list"] = animes["genre"].apply(lambda x: str(x).split(','))

In [ ]:
animes[animes.anime_id == 8074].rating

In [ ]:
print(mean_squared_error(predictions, labeled_ratings.rating.tolist()))


In [31]:
animes["type"] = animes["type"].astype('category')


,anime_id,name,genre,type,episodes,rating,members,genre_list
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572,"[Sci-Fi, Thriller]"
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266,"[Action, Comedy, Historical, Parody, Samur..."
5,32935,Haikyuu!!: Karasuno Koukou VS Shiratorizawa Ga...,"Comedy, Drama, School, Shounen, Sports",TV,10,9.15,93351,"[Comedy, Drama, School, Shounen, Sports]"
6,11061,Hunter x Hunter (2011),"Action, Adventure, Shounen, Super Power",TV,148,9.13,425855,"[Action, Adventure, Shounen, Super Power]"
7,820,Ginga Eiyuu Densetsu,"Drama, Military, Sci-Fi, Space",OVA,110,9.11,80679,"[Drama, Military, Sci-Fi, Space]"
8,15335,Gintama Movie: Kanketsu-hen - Yorozuya yo Eien...,"Action, Comedy, Historical, Parody, Samurai, S...",Movie,1,9.10,72534,"[Action, Comedy, Historical, Parody, Samur..."
9,15417,Gintama&#039;: Enchousen,"Action, Comedy, Historical, Parody, Samurai, S...",TV,13,9.11,81109,"[Action, Comedy, Historical, Parody, Samur..."
10,4181,Clannad: After Story,"Drama, Fantasy, Romance, Slice of Life, Supern...",TV,24,9.06,456749,"[Drama, Fantasy, Romance, Slice of Life, S..."
11,28851,Koe no Katachi,"Drama, School, Shounen",Movie,1,9.05,102733,"[Drama, School, Shounen]"
12,918,Gintama,"Action, Comedy, Historical, Parody, Samurai, S...",TV,201,9.04,336376,"[Action, Comedy, Historical, Parody, Samur..."


In [34]:
animes["genre_list"] = animes["genre"].apply(lambda x: str(x).split(','))

In [50]:
# enc = preprocessing.LabelBinarizer()
# # print(animes.type.cat.codes)
# enc.fit_transform(animes.type.cat.codes)

animes = pd.concat([animes,pd.get_dummies(animes.type)], axis=1).drop(["type"],1)

In [14]:
# pd.get_dummies(animes.genre_list)

In [63]:
animes.head()

,anime_id,name,episodes,rating,members,Movie,Music,ONA,OVA,Special,...,Slice of Life,Space,Sports,Super Power,Supernatural,Thriller,Vampire,Yaoi,Yuri,nan
0,32281,Kimi no Na wa.,1,9.37,200630,1,0,0,0,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5114,Fullmetal Alchemist: Brotherhood,64,9.26,793665,0,0,0,0,0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,28977,Gintama°,51,9.25,114262,0,0,0,0,0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,9253,Steins;Gate,24,9.17,673572,0,0,0,0,0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,9969,Gintama&#039;,51,9.16,151266,0,0,0,0,0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [53]:
g_list = animes.genre_list.tolist()
g_set = set()
for g in g_list:
    for x in g:
      g_str = x.strip()
      if g_str == "Music":
        # Music is also a type, avoid two columns with same name
        g_str = "Music_Genre"
      g_set.add(g_str)
#         print(type(x))



In [75]:
print(sorted(list(g_set)))
pd.get_dummies(list(g_set))

['Action', 'Adventure', 'Cars', 'Comedy', 'Dementia', 'Demons', 'Drama', 'Ecchi', 'Fantasy', 'Game', 'Harem', 'Hentai', 'Historical', 'Horror', 'Josei', 'Kids', 'Magic', 'Martial Arts', 'Mecha', 'Military', 'Music_Genre', 'Mystery', 'Parody', 'Police', 'Psychological', 'Romance', 'Samurai', 'School', 'Sci-Fi', 'Seinen', 'Shoujo', 'Shoujo Ai', 'Shounen', 'Shounen Ai', 'Slice of Life', 'Space', 'Sports', 'Super Power', 'Supernatural', 'Thriller', 'Vampire', 'Yaoi', 'Yuri', 'nan']


,Action,Adventure,Cars,Comedy,Dementia,Demons,Drama,Ecchi,Fantasy,Game,...,Slice of Life,Space,Sports,Super Power,Supernatural,Thriller,Vampire,Yaoi,Yuri,nan
0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
9,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [76]:
hot_labels_genres = pd.get_dummies(list(g_set))
def hot_label_sum(genres, hot_labels_genres = hot_labels_genres):
    hot_labels = np.zeros(len(hot_labels_genres["Action"]))
    
    for g in genres:
        g_str = g.strip()
        if g_str == "Music":
          # Music is also a type, avoid two columns with same name
          g_str = "Music_Genre"
        hot_labels += hot_labels_genres[g_str]
    return hot_labels


In [77]:
animes["genre_hot_labels"] = animes.genre_list.apply(lambda x: hot_label_sum(x))

AttributeError: 'DataFrame' object has no attribute 'genre_list'

In [78]:
g_df = pd.DataFrame(animes.genre_hot_labels.tolist(),columns=hot_labels_genres.keys())

AttributeError: 'DataFrame' object has no attribute 'genre_hot_labels'

In [79]:
animes = pd.concat([animes,g_df], axis=1).drop(["genre","genre_list", "genre_hot_labels"],1)

In [80]:
animes.dtypes

anime_id         float64
name              object
episodes          object
rating           float64
members          float64
Movie            float64
Music            float64
ONA              float64
OVA              float64
Special          float64
TV               float64
Action           float64
Adventure        float64
Cars             float64
Comedy           float64
Dementia         float64
Demons           float64
Drama            float64
Ecchi            float64
Fantasy          float64
Game             float64
Harem            float64
Hentai           float64
Historical       float64
Horror           float64
Josei            float64
Kids             float64
Magic            float64
Martial Arts     float64
Mecha            float64
                  ...   
Josei            float64
Kids             float64
Magic            float64
Martial Arts     float64
Mecha            float64
Military         float64
Music_Genre      float64
Mystery          float64
Parody           float64


In [81]:
# TODO: Episodes
animes.episode = animes.episodes.apply(lambda x: int(x) if x.strip() != "Unknown" else -1)

animes = animes.dropna()

animes["int_rating"] = animes.rating.apply(lambda x: round(x))

AttributeError: 'float' object has no attribute 'strip'

In [82]:
# Predict ratings within animes
def predict_on_animes_int_rating(animes,clf, test_size = 0.2):
    X = animes.drop(['anime_id','name','rating',"episodes", "int_rating"],1)
    y = animes.rating  
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
    
    clf.fit(X_train, y_train)
    print('Training Error: {:.3f}'.format(1 - clf.score(X_train, y_train)))
    print('Testing Error: %.3f' % (1 - clf.score(X_test, y_test)))

In [83]:
classifiers = [RandomForestRegressor(), GradientBoostingRegressor(), AdaBoostRegressor()]
# classifiers = [RandomForestClassifier(), GradientBoostingClassifier(), AdaBoostClassifier()]
for clf in classifiers:
    predict_on_animes_int_rating(animes,clf)
    print()

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [84]:
clf = xgb.XGBRegressor()
X = animes.drop(['anime_id','name','rating',"episodes", "int_rating"],1)
y = animes.rating
    
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    
clf.fit(X_train, y_train)
print('Training Error: {:.3f}'.format(1 - clf.score(X_train, 
                                                          y_train)))
print('Validation Error: {:.3f}'.format(1 - clf.score(X_test, 
                                                            y_test)))


NameError: name 'xgb' is not defined

In [85]:
print(len(list(animes.columns)), len(set(list(animes.columns))))
a_list = list(animes.columns)
for i in set(list(animes.columns)):1
  a_list.remove(i)
print(a_list)
     
    

IndentationError: unexpected indent (<ipython-input-85-157c820f5148>, line 4)

In [86]:
clf = xgb.XGBRegressor()
%timeit clf.fit(X_train, y_train)

NameError: name 'xgb' is not defined

In [87]:
%timeit GradientBoostingRegressor().fit(X_train, y_train)

NameError: name 'X_train' is not defined

In [88]:
animes.head()

,anime_id,name,episodes,rating,members,Movie,Music,ONA,OVA,Special,...,Slice of Life,Space,Sports,Super Power,Supernatural,Thriller,Vampire,Yaoi,Yuri,nan
0,32281.0,Kimi no Na wa.,1,9.37,200630.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5114.0,Fullmetal Alchemist: Brotherhood,64,9.26,793665.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,28977.0,Gintama°,51,9.25,114262.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,9253.0,Steins;Gate,24,9.17,673572.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,9969.0,Gintama&#039;,51,9.16,151266.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [89]:
a_file = open(DIR_PROCESSED + '/one_hot_encoded_anime.pickle','wb')
pickle.dump(animes, a_file)
a_file.close()

# Equivalent to above
with open(DIR_PROCESSED + '/one_hot_encoded_anime.pickle','wb') as a_file:
  pickle.dump(animes, a_file)

In [90]:
x = list(range(100))
y = np.arange(100)
def std_lib(x):
  return sum([i ** 2 for i in x])

def np_lib(x):
  return np.dot(x,x)





In [91]:
%timeit np_lib(y)

The slowest run took 29.04 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 1.22 µs per loop


In [92]:
%timeit std_lib(x)

10000 loops, best of 3: 37.3 µs per loop


In [93]:
print(np_lib(y), std_lib(x))

328350 328350


In [94]:
np_lib(y)

328350

In [95]:
std_lib(x)

328350

In [36]:
x = list(range(5))

In [35]:
print(x)

[0, 1, 2, 3, 4]


In [38]:
arr = [i**2 for i in x if i != 3]

[0, 1, 4, 16]

In [42]:
arr = []
for i in x:
    if i != 3:
        arr.append(i**2)
print(arr)

[0, 1, 4, 16]
